# Cardinality Constrained Portfolio Optimization

This notebook explores the **Cardinality-Constrained Mean-Variance (CCMV)** portfolio optimization problem, based on the methodology presented in *"Cardinality-Constrained Portfolio Optimization with Clustering"* (Ebrahimi et al., 2025).

The classical **Markowitz mean-variance model** is extended by adding a **cardinality constraint**, limiting the number of assets included in the portfolio. This transforms the problem into a **Mixed-Integer Quadratic Program (MIQP)**, making it significantly more challenging to solve due to its NP-hard nature.

In this notebook, we will:

- Formulate the classical and cardinality-constrained optimization problems
- Discuss the implications of binary variables and computational complexity
- Implement and solve small CCMV problems using Python
- Prepare for future integration of clustering methods as described in the paper

This work serves as both a theoretical deep dive and a practical foundation for reproducing the paper’s results.

### Portfolio Optimization Problem Formulation

The classical **mean-variance portfolio optimization** problem, introduced by Markowitz (1952), aims to minimize portfolio risk for a given level of expected return (or equivalently, balance both using a risk aversion parameter). It is formulated as:

$$
\min_{w \in \mathbb{R}^N} \quad w^\top \Sigma w - \gamma \mu^\top w \quad \text{subject to} \quad \mathbf{1}^\top w = 1,\quad 0 \leq w_i \leq 1 \quad \forall i \in \{1, \ldots, N\}
$$

where:

- $ w = (w_1, \ldots, w_N)^\top $ is the vector of portfolio weights  
- $ \Sigma \in \mathbb{R}^{N \times N} $ is the **covariance matrix** of asset returns  
- $ \mu \in \mathbb{R}^N $ is the **expected return vector**  
- $ \gamma > 0 $ is the **risk aversion coefficient**  
- $ \mathbf{1} \in \mathbb{R}^N $ is a vector of ones  
- The constraint $ \mathbf{1}^\top w = 1 $ ensures the portfolio is fully invested  
- The box constraint $ 0 \leq w_i \leq 1 $ limits individual asset weights

This is a **convex quadratic program** and can be efficiently solved using standard optimization methods.



This classical model is easy to solve due to its convexity, but it does not capture important practical constraints that investors often face. In real-world applications, it is common to:

- Reduce transaction costs  
- Control portfolio complexity  
- Avoid excessive concentration in a few assets

To reflect these needs, we introduce two key extensions:

#### 1. Cardinality Constraint

Investors may want to limit the total number of assets in the portfolio. Let $\Delta \in \mathbb{N}_+$ denote the maximum allowed number of assets. This leads to the **cardinality constraint**:

$$
\|w\|_0 \leq \Delta
$$

Here, $\|w\|_0$ is the **zero-norm**, which counts the number of nonzero entries in $w$. This constraint is **non-convex** and introduces **combinatorial complexity**, requiring the use of binary decision variables.

#### 2. Box Constraint on Asset Weights

To prevent overexposure to individual assets, we constrain each weight to lie within a maximum threshold. Let $\bar{w} \in (0, 1]$ be the maximum allowable weight per asset. The constraint is written as:

$$
w \in [0, \bar{w}]^N
$$

This **box constraint** ensures no single asset can dominate the portfolio, and also helps with regulatory compliance or risk diversification.

Together, these constraints transform the problem into a **Mixed-Integer Quadratic Program (MIQP)**, which is computationally much harder to solve than the unconstrained case.

We can then formulate the **Cardinality-Constrained Mean-Variance (CCMV)** optimization problem as follows:

$$
w^*(\Delta, \bar{w}) = \arg\min_{w \in \mathbb{R}^N} \quad w^\top \Sigma w - \gamma \mu^\top w \quad \text{subject to} \quad \mathbf{1}^\top w = 1, \quad 0 \leq w_i \leq \bar{w} \quad \forall i, \quad \|w\|_0 \leq \Delta
$$

Furthermore, it can be shown that this problem is NP-hard due to the combinatorial nature of the cardinality constraint. The introduction of binary variables to enforce this constraint complicates the optimization significantly.


### Solving the CCMV Problem

The **Cardinality-Constrained Mean-Variance (CCMV)** problem is computationally challenging due to its non-convex nature and the presence of binary decisions. To address this, we adopt the **clustering-based approach** proposed in the paper.

Specifically, we apply **hierarchical clustering** to group assets based on the similarity of their return profiles. Each cluster represents a group of highly correlated assets, and we solve the CCMV problem using the **cluster centroids** as representative assets.

This technique offers two key advantages:
- It **reduces the dimensionality** of the problem, making it more tractable.
- It **preserves key structure** in the data by exploiting the correlation between assets within each cluster.

By solving a reduced CCMV problem on the cluster level, we approximate the original high-dimensional problem while significantly improving computational efficiency.


### Implementation of Hierarchical Clustering 

We now implement the hierarchical clustering approach, following the pseudo-code provided in the paper. This algorithm clusters assets based on the similarity of their return profiles by analyzing the correlation structure of the return matrix.

The purpose of this step is to reduce the dimensionality of the portfolio optimization problem by grouping similar assets together. This allows us to simplify the later stages of solving the CCMV problem, while still preserving the core structure of the asset universe.

The clustering is performed using a bottom-up (agglomerative) approach, with distances computed between cluster centroids in the distance space. The final partition is determined by cutting the dendrogram at the optimal level, based on the largest gap in merge distances.

In [2]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [3]:
def Hierarchical_Clustering(R: np.ndarray) -> np.ndarray:
	"""
	Perform hierarchical clustering on the given return matrix R.
	
	Parameters:
	R (np.ndarray): A return matrix.
	
	Returns:
	np.ndarray: A 1D array of clusters.
	"""

	# 1. Compute the correlation amtrix tau
	tau = np.corrcoef(R, rowvar=False)

	# 2. Construct the distance matrix D
	D = 1-tau

	# 3. Initialize clusters C
	C = [[i] for i in np.arange(R.shape[1])]

	def d(Ci, Cj):
		"""
		Compute the distance between two clusters Ci and Cj.
		
		Parameters:
		Ci (list): First cluster.
		Cj (list): Second cluster.
		
		Returns:
		float: The distance between the two clusters.
		"""
		cent_i = 1/ len(Ci) * np.sum(D[Ci, :], axis=0)
		cent_j = 1/ len(Cj) * np.sum(D[Cj, :], axis=0)
		dist = (len(Ci) * len(Cj)) / (len(Ci) + len(Cj)) * np.linalg.norm(cent_i - cent_j) ** 2
		return dist
	# 4. Repeat until only one cluster remains
	merge_distances = []
	merge_log = []
	while len(C) > 1:
		min_distance = float('inf')
		best_pair = (0, 0)
		for i in range(len(C)):
			for j in range(i+1, len(C)):
				distance = d(C[i], C[j])
				if distance < min_distance:
					min_distance = distance
					best_pair = (i, j)
		
		# Merge the best pair of clusters
		i, j = best_pair
		new_cluster = C[i] + C[j]
		merge_log.append((C[i], C[j], new_cluster, min_distance))
		del C[max(i, j)]  # Remove the larger index first to avoid index issues
		del C[min(i, j)]
		C.append(new_cluster)
		merge_distances.append(min_distance)

	# 5. Determine optimal number of clusters
	gap = []
	for k in range(1, len(merge_distances)):
		gap.append(merge_distances[k] - merge_distances[k-1])
	largest_gap = np.argmax(gap) + 1
	threshold = merge_distances[largest_gap]
	# 6. Assign clusters based on the threshold
	clusters = [{i} for i in range(R.shape[1])]

	for log in merge_log:
		if log[3] <= threshold:
			Ci, Cj = log[0], log[1]
			to_merge = []
			for cluster in clusters:
				if any(i in cluster for i in Ci) or any(i in cluster for i in Cj):
					to_merge.append(cluster)

			if len(to_merge) == 2:
				clusters.remove(to_merge[0])
				clusters.remove(to_merge[1])
				clusters.append(set(to_merge[0]).union(set(to_merge[1])))
	
	labels = np.empty(R.shape[1], dtype=int)
	for cluster_id, asset_set in enumerate(clusters):
		for asset in asset_set:
			labels[asset] = cluster_id
	return labels

### Modeling the Clustered CCMV Problem

Let $C = \{C_1, \dots, C_K\}$ be the optimal clusters obtained from hierarchical clustering. We now introduce three key parameters:

- Maximum allowable weight for any asset $\bar{w}$
- Maximum number of assets per cluster $\Delta = (\Delta_1, \dots \Delta_k)$
- $\alpha = (\alpha_1, \dots, \alpha_k)$, where $\alpha_i$ is the fraction of the portfolio allocated to cluster $C_i$

We can then define the feasible set of portfolio weights accounting for both clustering and cardinality constraints:
$$
	\mathcal{A}_C(\Delta, \alpha, \bar{w}) = \left\{ w \in [0, \bar{w}]^N : \|w_{C_i}\|_0 \leq \Delta_i, \quad \mathbf{1}^\top w_{C_i} = \alpha_i \quad \forall i = 1, \ldots, K \right\}

$$

We can then express the clustered CCMV problem as:

$$
	w^*(C, \Delta, \alpha, \bar{w}) = \arg\min_{w \in \mathcal{A}_C(\Delta, \alpha, \bar{w})} \quad w^\top \Sigma w - \gamma \mu^\top w
$$

Our goal is to determine the optimal cluster-level allocation $\alpha_k$, the maximum number of assets $\Delta_k$ per cluster $C_k$, and the optimal asset weights $w$ to balance allocation and enchance portfolio diversification.

We now introduce an intra-cluster optimization problem to determine the optimal asset weights within each cluster. This is formulated as:
$$
	w^*_{C_k} = \arg\min_{w \in \mathcal{A}_{C_k}(\Delta_k, \alpha_k, \bar{w})} \quad w^\top \Sigma_{C_k} w - \gamma \mu_{C_k}^\top w
$$

Where $\Sigma_{C_k}$ and $\mu_{C_k}$ are the covariance matrix and expected return vector for the assets in cluster $C_k$. We can also define the feasible set used above as:
$$
\mathcal{A}_{C_k}(\Delta_k, \alpha_k, \bar{w}) = \left\{ w \in [0, \bar{w}]^{|C_k|} : \|w\|_0 \leq \Delta_k, \quad 1^Tw = \alpha_k \right\}.
$$

This means that we can write the original feasible set as:
$$
\mathcal{A}_C(\Delta, \alpha, \bar{w}) = \bigcap_{k=1}^K \left\{ w : w_{C_k} \in \mathcal{A}_{C_k}(\Delta_k, \alpha_k, \bar{w}) \right\}.
$$

We now look at the objective function. To simplify things, we neglect correlations between different clusters, which allows us to write the objective function as a sum of intra-cluster objectives:
$$
f(w) \approx \sum_{k=1}^K f_{C_k}(w) = \sum_{k=1}^K \left( \sum_{i,j \in C_k} w_iw_j\sigma_{ij} -\gamma \sum_{i \in C_k} w_i \mu_i \right).
$$

We can then approximate the optimal objective value as 
$$
f^*_C(\Delta, \alpha) \approx \sum_{k=1}^K f^*_{C_k}(\Delta_k, \alpha_k)
$$

where $f^*_C(\Delta, \alpha) = \min_{w \in \mathcal{A}_C(\Delta, \alpha, \bar{w})} f(w)$ and $f^*_{C_k}(\Delta_k, \alpha_k) = \min_{w_{C_k} \in \mathcal{A}_{C_k}(\Delta_k, \alpha_k, \bar{w})} f_{C_k}(w)$.

We can then finally define

$$
	f^*_C(\Delta) := \min_{\substack{\Delta, \alpha, w_i; \\ \sum_{k=1}^K \Delta_k = \Delta; \\ \sum_{k=1}^K \alpha_k = 1}} f^*_C(\Delta, \alpha) = \min_{\substack{\Delta, \alpha, w_i; \\ \sum_{k=1}^K \Delta_k = \Delta; \\ \sum_{k=1}^K \alpha_k = 1}} \min_{w \in \mathcal{A}_C(\Delta, \alpha, \bar{w})} f(w)
$$

This final formulation represents a two-level optimization: the outer minimization selects how to allocate weight ($\alpha$) and sparsity ($\Delta$) across clusters, while the inner minimization solves for the optimal asset weights within each cluster under those constraints. Together, this balances diversification across clusters with precise selection within them.


### Implementation of $\Delta$-CCMV and $\alpha$-CCMV

We now implement the $\Delta$-CCMV and $\alpha$-CCMV problems as described by the pseuducode in the paper. The first method $\Delta$-CCMV pre-assigns the number of selected assets per cluster, while the second method $\alpha$-CCMV pre-assigns the fraction of the portfolio allocated to each cluster.

#### $\Delta$-CCMV

We introduce a score based method to determine the number of selected assets per cluster. For each cluster $C_k$, we compute a score based on the expected return and risk of the assets within that cluster.
We assign $\Delta_k = \lfloor{\Delta \times s_k}\rfloor$ to each cluster $C_k$ where $s_k$ is the relative importance of the cluster. In the algorithm, we compute the score as follows:
$$
s_k = \left(\frac{\gamma \mu_{C_k} - \lambda}{2 \sigma^2_{C_k}}\right)_+
$$

where $\mu_{C_k}$ and $\sigma^2_{C_k}$ are the expected return and variance of the assets in cluster $C_k$. $\lambda$ is a normalization constant that ensures the scores sum to 1.

By using scores, we can ensure that the clusters with stronger risk-adjusted preformance are more represented in the final portfolio, while still respecting the cardinality constraint.

In [4]:
from scipy.optimize import brentq
from math import floor
import cvxpy as cp
import numpy as np

def allocate_cardinality(clusters, mu, Sigma, Delta, gamma):
    """
    Allocate the number of assets (Delta_k) to each cluster based on the cluster-level 
    expected return and variance, as defined in Step 2 of the Δ-CCMV algorithm.

    Parameters:
    clusters (list of lists): Each sublist contains indices of assets in a cluster.
    mu (np.ndarray): Vector of expected returns for all assets (length N).
    Sigma (np.ndarray): Covariance matrix of asset returns (shape N x N).
    Delta (int): Total number of assets to include in the portfolio (cardinality constraint).
    gamma (float): Risk aversion parameter.

    Returns:
    list of int: A list Delta_k of length K (number of clusters), where Delta_k[i] is the 
    number of assets allocated to cluster i. The sum of all Delta_k equals Delta.
    """

    # Step 1: Compute cluster-level expected returns and variances
    cluster_returns = []
    cluster_variances = []
    for cluster in clusters:
        cluster_assets = np.array(cluster)
        cluster_mu = mu[cluster_assets]
        cluster_Sigma = Sigma[np.ix_(cluster_assets, cluster_assets)]
        
        expected_return = np.mean(cluster_mu)
        variance = np.mean(np.diag(cluster_Sigma))
        
        cluster_returns.append(expected_return)
        cluster_variances.append(variance)
    
    # Step 2: Define the score function
    def score_function(lambda_, mu_k, sigma2_k, gamma):
        scores = []
        for mu_i, sigma2_i in zip(mu_k, sigma2_k):
            raw = (gamma * mu_i - lambda_) / (2 * sigma2_i)
            scores.append(max(0, raw))
        return scores

    def root_objective(lambda_, mu_k, sigma2_k, gamma):
        scores = score_function(lambda_, mu_k, sigma2_k, gamma)
        return sum(scores) - 1

    # Step 3: Solve for lambda*
    lambda_star = brentq(root_objective, -1000, 1000, args=(cluster_returns, cluster_variances, gamma))

    # Step 4: Final scores and allocation
    scores = score_function(lambda_star, cluster_returns, cluster_variances, gamma)
    Delta_k = [floor(Delta * s) for s in scores]

    return Delta_k

def solve_intracluster_miqp(mu_cluster, Sigma_cluster, delta_k, alpha_k, bar_w, gamma):
    """
    Solve the MIQP problem for a single cluster C_k.

    Parameters:
    mu_cluster (np.ndarray): Expected returns for assets in the cluster (shape: (n,))
    Sigma_cluster (np.ndarray): Covariance matrix for the cluster (shape: (n, n))
    delta_k (int): Number of assets to select in this cluster.
    alpha_k (float): Fraction of the total portfolio weight allocated to this cluster.
    bar_w (float): Maximum allowed weight per asset.
    gamma (float): Risk aversion parameter.

    Returns:
    np.ndarray: Optimal weights for the assets in the cluster (length n).
    """
    n = len(mu_cluster)  # number of assets in the cluster

    # Step 1: Define variables
    w = cp.Variable(n)
    z = cp.Variable(n, boolean=True)

    # Step 2: Define the objective function
    objective = cp.Minimize(cp.quad_form(w, Sigma_cluster) - gamma * mu_cluster @ w)

    # Step 3: Define the constraints
    constraints = [
        cp.sum(w) == alpha_k,
        w >= 0,
        w <= bar_w * z,
        cp.sum(z) <= delta_k
    ]

    # Step 4: Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.GUROBI if cp.GUROBI in cp.installed_solvers() else cp.ECOS_BB)

    # Step 5: Return the solution
    return w.value
	

def delta_ccmv(R, mu, Sigma, Delta, bar_w, gamma):
	"""
	Solve the CCMV problem by pre-assigning the number of assets in each cluster.
	
	Parameters:
	R (np.ndarray): A return matrix.
	mu (np.ndarray): Expected returns.
	Sigma (np.ndarray): Covariance matrix of returns.
	Delta (int): Total number of assets to include in the portfolio (cardinality constraint)
	bar_w (float): Maximum allowed weight per asset
	gamma (float): Risk aversion parameter.
	
	Returns:
	w: Optimal portfolio weights
	alpha: cluster allocation proportions
	"""

	# Step 1: Collect the clusters from R
	clusters = Hierarchical_Clustering(R)

	# Step 2: Delta pre-assignment
	
	Delta_k = allocate_cardinality(clusters, Delta)
     
	# Step 3: Optimization
     
	alpha_k = [1 / len(clusters) for _ in clusters]
	w = np.zeros(R.shape[1])
	for i, cluster in enumerate(clusters):
		mu_cluster = mu[cluster]
		Sigma_cluster = Sigma[np.ix_(cluster, cluster)]
		delta_k = Delta_k[i]
		alpha_k_val = alpha_k[i]
		
		w_cluster = solve_intracluster_miqp(mu_cluster, Sigma_cluster, delta_k, alpha_k_val, bar_w, gamma)
        
		for j, asset_idx in enumerate(cluster):
			w[asset_idx] = w_cluster[j]
	return w, alpha_k
